In [ ]:
import os
import itertools
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from odors import load_hallem, normalize_chem_ids
import chemutils as cu
import seaborn as sns
from drosolf import orns
sns.set()

In [ ]:
# Load hallem + IR dataset
os.chdir('D://research//ejhonglab//natural_odors-master')
n_odors = 5
hh = load_hallem()
hh = pd.read_csv('../data/orirdf.csv')
hh = hh.set_index('inchi')
del hh['name']

In [ ]:
# Load Hallem dataset without baseline; run this if you want hallem dataset instead of Hallem + IR
# Copied from odors.py by Tom 
def load_hallem(norm_chem_ids=True, chem_id_in_cols=False, hallem_classes=False,
    **convert_kwargs):
    """Returns DataFrame with Hallem & Carlson data, with normalized chem ids.

    If `chem_id_in_cols` is False, it is left as the index, rather than moved to
    a column.
    """
    chem_id = 'inchi'
    
    import drosolf.orns as orns
    hallem_df = orns.orns(add_sfr = False)

    # From 'odor'. To be consistent w/ output of other load_* fns.
    hallem_df.index.name = 'name'

    # TODO probably move chemical class stuff to drosolf...
    if hallem_classes:
        chemical_classes = []
        idx = 0
        for odor_name in hallem_df.index:
            next_idx = idx + 1
            if (next_idx < len(first_hallem_odor_and_chemical_class) and
                odor_name == first_hallem_odor_and_chemical_class[next_idx][0]):
                idx = next_idx

            curr_class = first_hallem_odor_and_chemical_class[idx][1]
            chemical_classes.append(curr_class)
        hallem_df['hallem_class'] = chemical_classes

    if norm_chem_ids:
        if chem_id_in_cols:
            hallem_df = hallem_df.reset_index()

        orig_shape = hallem_df.shape
        # TODO TODO am i breaking comparison to hallem by squashing extra inchi
        # info?? how to handle overlapping cases? do i actually have chemicals
        # detected which have multiple stereoisomers in hallem data?
        # NOTE: strip_inchi_stereochem flag does not seem to affect anything
        # that doesnt go through load_shimadzu, as-is
        hallem_df = cu.convert(hallem_df, to_type=chem_id,
            **convert_kwargs
        )

        # TODO delete after adding flag to convert to check 1:1 on converted and
        # orig IDs + using that here
        if chem_id_in_cols:
            duped = hallem_df[chem_id].duplicated(keep=False)
        else:
            duped = hallem_df.index.duplicated(keep=False)
        '''
        if duped.any():
            print('Hallem names that mapped to the same {}:'.format(chem_id))
            duped_names = hallem_df.index[duped]
            print(duped_names)
            duped_chems = chemutils.convert(duped_names, to_type=chem_id,
                ignore_cache=True, verbose=True
            )
        '''
        assert not duped.any()
        #

        if chem_id_in_cols:
            assert hallem_df.shape[0] == orig_shape[0]
            assert hallem_df.shape[1] > orig_shape[1]
        else:
            assert hallem_df.shape == orig_shape

    return hallem_df

hh = load_hallem()

In [ ]:
def get_inchi(name):
    inchi = cu.name2compound(name).inchi
    return inchi.replace('InChI=', '')

# plot correlation matrix
def plot_corr_mat(hh, candidate, figname, plot_all):
    candidate_inchi = {inchi:cu.convert(inchi, from_type='inchi', to_type='name', allow_nan=True) for inchi in candidate}
    in_hallem = [ii for ii in candidate if ii in hh.index]
    
    candidate_hallem = hh.loc[in_hallem, :]
    candidate_corr = candidate_hallem.T.corr()
    candidate_corr = candidate_corr.rename(columns = candidate_inchi, index = candidate_inchi)
    candidate_corr.columns.name = None
    candidate_corr.index.name = None
    
    if (candidate_corr>0).all(axis=None) or plot_all:
        #fig, ax = plt.subplots(figsize=(12,8)) 
        sns.heatmap(candidate_corr, annot = True)
        plt.xticks(rotation=90) 
        plt.yticks(rotation=0) 
        plt.tight_layout()
        plt.savefig('../results/' + str(n_odors) + '_' + figname + 'HallemIR_final.png')
    
        plt.clf()
    return

# plot pmi matrix
def plot_pmi(indf2, candidate, figname):
    candidate_inchi = {inchi:cu.convert(inchi, from_type='inchi', to_type='name', allow_nan=True) for inchi in candidate}
    df = pd.DataFrame()
    for i in itertools.combinations(candidate, 2):
        try:
            df.loc[i[0], i[1]] = indf2.loc[i, 'pointwise_mutual_information']
            df.loc[i[1], i[0]] = df.loc[i[0], i[1]]
        except:
            df.loc[i[0], i[1]] = indf2.loc[i[::-1], 'pointwise_mutual_information']
            df.loc[i[1], i[0]] = df.loc[i[0], i[1]]
    df = df.rename(columns = candidate_inchi, index = candidate_inchi)
    df = df[df.index]
    
    #fig, ax = plt.subplots(figsize=(12,8)) 
    sns.heatmap(df, annot = True)
    plt.xticks(rotation=90) 
    plt.yticks(rotation=0) 
    plt.tight_layout()
    plt.savefig('../results/' + str(n_odors) + '_' + figname + 'pmi.png')

    plt.clf() 
    return df

In [ ]:
# filter out large pmi
indf = pd.read_csv('../data/indf2.csv')
indf = indf[indf['pointwise_mutual_information']<0.4]
indf = indf[indf['hallem_correlation'].notnull()]
# indf = indf[indf['hallem_correlation_orir'].notnull()]

odor_pair_set = set([tuple(row) for row in indf[['inchi1', 'inchi2']].values])
odor_list = np.unique(indf[['inchi1', 'inchi2']].values.flatten())

In [ ]:
drop = ['dimethyl sulfide', # unpleasant to work with
        'acetoin', # in fly food mixture
        'ethanol',
        'acetic acid',
        'propanoic acid, 2-methyl-',
        'propanoic acid',
        'alpha-terpinolene', # not soluble in water
        'alpha-pinene',
        'hexanal',
        '(e)-2-hexenal',
        'linalool',
        'pentanal',
        'methyl hexanoate',
        'butanal',
        'octanol', 
        'ethyl hexanoate', 
        'ethyl butyrate', 
        'isoamyl acetate',
        '1-hexanol',
        '2-butenoic acid, ethyl ester, (E)-',
        '2-methylpropyl acetate',
        'heptan-2-one',
        'hexyl acetate',
        'methyl butanoate',
        'ethyl benzoate',
        'ethanal', # high vapor pressure
        'furfural', # already present in control 1
        ]

drop = [
        '1S/C2H6S/c1-3-2/h1-2H3',
        '1S/C4H8O2/c1-3(5)4(2)6/h3,5H,1-2H3',
        '1S/C2H6O/c1-2-3/h3H,2H2,1H3',
        '1S/C2H4O2/c1-2(3)4/h1H3,(H,3,4)',
        '1S/C4H8O2/c1-3(2)4(5)6/h3H,1-2H3,(H,5,6)',
        '1S/C3H6O2/c1-2-3(4)5/h2H2,1H3,(H,4,5)',
        '1S/C10H16/c1-8(2)10-6-4-9(3)5-7-10/h4H,5-7H2,1-3H3',
        '1S/C10H16/c1-7-4-5-8-6-9(7)10(8,2)3/h4,8-9H,5-6H2,1-3H3',
        '1S/C6H12O/c1-2-3-4-5-6-7/h6H,2-5H2,1H3',
        '1S/C6H10O/c1-2-3-4-5-6-7/h4-6H,2-3H2,1H3/b5-4+',
        '1S/C10H18O/c1-5-10(4,11)8-6-7-9(2)3/h5,7,11H,1,6,8H2,2-4H3',
        '1S/C5H10O/c1-2-3-4-5-6/h5H,2-4H2,1H3',
        '1S/C7H14O2/c1-3-4-5-6-7(8)9-2/h3-6H2,1-2H3',
        '1S/C4H8O/c1-2-3-4-5/h4H,2-3H2,1H3',
        '1S/C8H18O/c1-2-3-4-5-6-7-8-9/h9H,2-8H2,1H3',
        '1S/C8H16O2/c1-3-5-6-7-8(9)10-4-2/h3-7H2,1-2H3',
        '1S/C6H12O2/c1-3-5-6(7)8-4-2/h3-5H2,1-2H3',
        '1S/C7H14O2/c1-6(2)4-5-9-7(3)8/h6H,4-5H2,1-3H3',
        '1S/C6H14O/c1-2-3-4-5-6-7/h7H,2-6H2,1H3',
        '1S/C6H10O2/c1-3-5-6(7)8-4-2/h3,5H,4H2,1-2H3/b5-3+',
        '1S/C6H12O2/c1-5(2)4-8-6(3)7/h5H,4H2,1-3H3',
        '1S/C7H14O/c1-3-4-5-6-7(2)8/h3-6H2,1-2H3',
        '1S/C8H16O2/c1-3-4-5-6-7-10-8(2)9/h3-7H2,1-2H3',
        '1S/C5H10O2/c1-3-4-5(6)7-2/h3-4H2,1-2H3',
        '1S/C9H10O2/c1-2-11-9(10)8-6-4-3-5-7-8/h3-7H,2H2,1H3',
        '1S/C2H4O/c1-2-3/h2H,1H3',
        '1S/C5H4O2/c6-4-5-2-1-3-7-5/h1-4H',
        ]

In [ ]:
# Select candidate combinations
n_odors = 5

retval = []
for candidate in itertools.combinations(odor_list, n_odors):
    out = True
    
    for ii in itertools.combinations(candidate, 2):
        if ii not in odor_pair_set and ii[::-1] not in odor_pair_set:
            out = False
            break

    if out == True and not bool(set(candidate) & set(drop)):
        retval.append(list(candidate))
    
repeat = []
for ii in itertools.combinations(retval, 2):
    if set(ii[0]) == set(ii[1]):
        repeat.append(ii[0])

unique_ret = [ii for ii in retval if ii not in repeat]

In [ ]:
# Filter out insoluble
water_solubility = pd.read_csv('../data/water_solubility.csv')
water_solubility = water_solubility.set_index('inchi')

unique_ret2 = unique_ret[:]
for candidate in unique_ret2:
    print(candidate)
    for inchi in candidate:
        try:
            if inchi not in water_solubility.index:
                sol = cu.inchi2water_solubility(inchi)
                ws = sol['water_solubility']
                water_solubility.loc[inchi, :] = sol.values
            else:
                ws = water_solubility.loc[inchi, 'water_solubility']
                
            if ws is None or math.isnan(ws) or ws < 5:
                unique_ret.remove(candidate)
                break
        except:
            unique_ret.remove(candidate)
            break

water_solubility.to_csv('../data/water_solubility.csv')

In [ ]:
# Plot Hallem correlation matrix for selected candidates
n_odors = 2
for idx in range(len(unique_ret)):
    candidate = unique_ret[idx]
    plot_corr_mat(hh, candidate, str(idx), True)

In [ ]:
# Plot pointwise mutual information for candidates
indf = pd.read_csv('../data/indf2.csv')
indf2 = indf.set_index(['inchi1', 'inchi2'])

idx_list = range(len(unique_ret))
for idx in idx_list:
    candidate = unique_ret[idx]
    print(candidate)
    plot_pmi(indf2, candidate, str(idx))

In [ ]:
# Plot Hallem correlation matrix for a given list of odors

mixture = ['ethanol', 'acetic acid', 'propanoic acid, 2-methyl-', 'propanoic acid'] # flyfood (acetoin not in Hallem)
# mixture = ['valeric acid', 'methyl salicylate', 'furfural', '1-octen-3-ol', '2-heptanone'] # control1
# mixture = ['ethyl acetate', 'ethyl butyrate', 'isoamyl alcohol', 'ethanol', 'isoamyl acetate'] # kiwi
# mixture = ['benzaldehyde', 'cis-3-hexenol', 'ethyl acetate', '3-methylthio-1-propanol', 'butanoic acid'] # control2
mixture = [get_inchi(name) for name in mixture]
plot_corr_mat(hh, mixture, 'flyfood', True)

In [ ]:
# Plot pointwise mutual information (pmi) for a given list of odors

indf = pd.read_csv('../data/indf2.csv')
indf2 = indf.set_index(['inchi1', 'inchi2'])

mixture = ['acetoin', 'ethanol', 'acetic acid', 'propanoic acid, 2-methyl-', 'propanoic acid'] # flyfood
# mixture = ['valeric acid', 'methyl salicylate', 'furfural', '1-octen-3-ol', '2-heptanone'] # control1
# mixture = ['ethyl acetate', 'ethyl butyrate', 'isoamyl alcohol', 'ethanol', 'isoamyl acetate'] # kiwi
# mixture = ['benzaldehyde', 'cis-3-hexenol', 'ethyl acetate', '3-methylthio-1-propanol', 'butanoic acid'] # control2
mixture = [get_inchi(name) for name in mixture]

df = plot_pmi(indf2, mixture, 'flyfood_allsamples')